In [ ]:
import pandas as pd 
import numpy as np

import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")


##데이터 불러오기


student = pd.read_csv('/content/drive/MyDrive/명륜이의 서재/DATA/1209_final_data/1209_data.csv')
student.head()
data = pd.read_csv('/content/drive/MyDrive/명륜이의 서재/DATA/1209_final_data/1211_scraping_data.csv')
data.head()

,book_id,img,pub_year,subject,page
0,EE0414400,Y,2010.0,요루와 휘린의 완벽한 결혼 :김문숙 소설,232.0
1,EM0831764,Y,2009.0,"26EURO :가난한, 그러나 살아있는 219일간의 무전여행기",339.0
2,EM0670351,Y,2004.0,한국 권력구조의 이해,394.0
3,EM0898861,Y,2012.0,동물 해방,504.0
4,EM0698235,Y,2005.0,코리아 다시 생존의 기로에 서다,463.0


## 1. Title Preprocess

In [2]:
#preprocess 
## title :: nbsp --> 공백으로 바꾸기

# checking
# data[data['subject'].str.contains('nbsp', regex = False) == True][:50]

# remove nbsp
def remove_nbsp(x):
    try:
        x = x.replace('&nbsp;', ' ')
        # return x
    except:
        # return x
        pass

    try: 
      x = x.split('=')
      return x[0]
    except:
      return x


# data['subject'] = data['subject'].apply(lambda x : remove_nbsp(x))
# data

'파리 예술카페 기행 '

## 2. Clean Page


In [ ]:
## clear page nums

def clean_page(x):
    if x == "notfound":
        return x
    else: 
        try: 
            float(x)
            return float(x)
        
        except: 
            if 'p' in x:
                new_x = x.split('p')
                try:
                    return float(new_x[0].split(' ')[-1])
                except:
                    try:
                        return float(new_x[1].split(' ')[0])
                    except:
                        try:
                            return float(x.split(' p')[0].split(' ')[0])
                        except:
                            try: 
                                return float(x.split(' p')[0].split(' ')[1])
                            except:
#                             print(x)
                                return np.nan
                
        
            elif ('cm' in x) or ('책' in x) or ('冊' in x) : 
#             print(x)
                return np.nan
            else:
                if ',' in x:
                    new_x = x.split(',')
                    if 'p' in x:
                        try:
                            return float(x.split('p')[0])
                        except:
#                         print(x)
                            return np.nan
                    
                
                    elif len(new_x) == 2:
                        try:
                            return float(new_x[0])
                        except:
                            try:
                                return float(new_x[1])
                            except:
                                if '.' in new_x[1]:
                                    try: 
                                        return float(new_x[1].split('.')[0])
                                    except:
#                                     print(x)
                                        return np.nan                                
                                elif ';' in new_x[1]:
                                    try:
                                        return float(new_x[1].split(';')[0])
                                    except:
#                                     print(x)
                                        return np.nan
                                else:
#                                 print(x)
                                    return np.nan
               
                    elif len(new_x) == 3:
                        try: 
                            return float(new_x[1])
                        except:
                            try:
                                return float(new_x[2])
                            except:
                                try:
                                    return float(new_x[0])
                                except:
#                                 print(x)
                                    return np.nan
                
               
                elif ';' in x:
                    try:
                        return float(x.split(';')[-1])
                    except:
                        if 'p' in x:
                            try:
                                return float(x.split('p')[0])
                            except:
                                return np.nan
                        else:
                            return np.nan


## 정제 과정에서 페이지수가 10이하로 잘못 정제된 것들 체크 --> 삭제 혹은 평균값 대체 해야 할듯..!
def check_under_10(x):
    try:
        if x['page'] < 10:
            print(x['book_id'], x['page'])
    except:
        pass 

# data['page'] = data['page'].apply(lambda x : clean_page(x))

# data.apply(lambda x : check_under_10(x), axis = 1)


## 3. Clean Pub Year


In [ ]:
#preprocess ddc
#CLEAN PUB YEAR 

def clean_pub_year(x):
    
    try: 
        float(x)
        return float(x)
    
    except:
        # print(x)
        r = list(x)        
        s=''
        for i in r:
            if i.isdigit():
                s+=str(i)
            else:
                pass
            
        #for years which have duration (first year)
        # print(s)
        s = s[:4]
        # print('The year is now.. '+str(s))
        #for misinfo 
        if s == '':
            if x == 'notfound':
              return np.nan
            else: 
              # print(x)
              #발행년 미상 다수 :Köln :Taschen. 민음사 [발행년미상] 서울 :淸文閣 [발행년 불명]. [n. d.] Amsterdam :BIS 등
              return np.nan
        else:
            return float(s)
                
        

# data['pub_year'] = data['pub_year'].apply(lambda x: clean_pub_year(x))
        
# clean_pub_year('ad')

# data.head()
data.isnull().sum()
# data.loc[data['pub_year'].isnull()][:50]

book_id     0
img         0
pub_year    0
subject     0
page        0
dtype: int64

## 4. Clean DDC

## 5. Match DDC in book table

In [ ]:
# student.shape #(605496, 10)
def student_ddc(student):
  book = student.drop_duplicates(['book_id']) #195636,2
  book = book[['book_id', 'ddc']]
  book_id_ddc= book.set_index('book_id').to_dict('dic')
  book_id_ddc= book_id_ddc['ddc'] 
  return book_id_ddc

# book_id_ddc= student_ddc(student)

In [ ]:
def ddc_match(x):
  if x in book_id_ddc:
    return book_id_ddc[x]
  else:
    return 'notfound'

def ddc_sort(x):

  import re 
  # print(x)

  if x == 'notfound':
    # print(x)
    return np.nan
    
  else: 
    #for words in front of the ddc 
    x= re.sub(r'\D','',x)
    # print(x)
  
    ddc = x[:3]
    # print(ddc)
  
    ddc = int(ddc)
    first = ddc//100
  
    return first
  


# data['ddc'] = data['book_id'].apply(lambda x: ddc_match(x))
# data['ddc'] = data['ddc'].apply(lambda x: ddc_sort(x))

In [ ]:
if __name__ == "__main__": 
  print('start preprocessing subject/page/pub_year.........................')
  #clean subject title
  data['subject'] = data['subject'].apply(lambda x : remove_nbsp(x))
  #clean page 
  data['page'] = data['page'].apply(lambda x : clean_page(x))
  #clean pub year
  data['pub_year'] = data['pub_year'].apply(lambda x: clean_pub_year(x))

  print('start preprocessing ddc.........................')
  #make ddc dictionary by student table 
  book_id_ddc= student_ddc(student)
  #clean ddc for book table
  print('start applying ddc on book data.........................')
  data['ddc'] = data['book_id'].apply(lambda x: ddc_match(x))
  data['ddc-short'] = data['ddc'].apply(lambda x: ddc_sort(x))
  #match ddc for book table 

start preprocessing subject/page/pub_year.........................
start preprocessing ddc.........................
start applying ddc on book data.........................


In [ ]:
data.head(20)
data.isnull().sum()
data[data['ddc-short'].isnull()]

#ddc nan = ddc that is not found in the student book dataset 
#pub year nan = pub year not found .. weird pubyear is removed 


,book_id,img,pub_year,subject,page,ddc,ddc-short
104232,EM0738392,Y,2006.0,벼랑 끝에서 만나는 처칠 :패배자의 인생역전 스토리,212,notfound,NaN
104233,EM0680978,Y,2004.0,우리는 결코 실패하지 않는다 :돌파의 CEO 윈스턴 처칠,342,notfound,NaN
104634,EM0847418,Y,2006.0,영화 읽기의 즐거움,415,notfound,NaN
105106,EE0448683,Y,2013.0,"쥰페이, 다시 생각해!",323,notfound,NaN
105293,EE0479989,Y,2016.0,전기의 역사,263,notfound,NaN
...,...,...,...,...,...,...,...
166542,EM0987258,Y,2016.0,장애인복지의 이해 =Social welfare for disabled,327,notfound,NaN
166566,EE0487516,Y,2017.0,(한 권으로 끝내는) 셰익스피어,1247,notfound,NaN
166568,EM0269534,N,1987.0,한국전쟁과 한미관계 :1943-1953,389,notfound,NaN
166584,EE0255478,N,1997.0,Mary Kay :당신은 모든 것을 소유할 수 있다,335,notfound,NaN


In [ ]:
student[student['book_id']=='EM0738392']

,student_id,major,book_id,ddc,title,meta_id,borrow_type,b_date,r_date,return_type
